In [1]:
import keras
import keras.preprocessing.image
from keras_retinanet.models.resnet import custom_objects
from keras_retinanet.preprocessing.csv_generator import CSVGenerator

import imageio

imageio.plugins.ffmpeg.download()

from moviepy.editor import VideoFileClip
import PIL.Image
import time
import numpy as np
import cv2

Using TensorFlow backend.
/home/appsyoon/miniconda3/envs/capstone/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
class VideoProcessor():
    def __init__(self):
        self.model = keras.models.load_model('./resnet50_csv_20.h5', custom_objects=custom_objects)
        # create image data generator object for the preprocessing functionality
        generator = keras.preprocessing.image.ImageDataGenerator()
        self.csv_generator = CSVGenerator(
            './test.csv',
            './classes.csv',
            generator,
            batch_size=1,
        )
        
    def process_image(self, image):
        # copy to draw on
        draw = image.copy()

        # preprocess image for network
        image = self.csv_generator.preprocess_image(image)
        image, scale = self.csv_generator.resize_image(image)

        # process image
        start = time.time()
        _, _, detections = self.model.predict_on_batch(np.expand_dims(image, axis=0))

        # compute predicted labels and scores
        predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
        scores = detections[0, np.arange(detections.shape[1]), 4 + predicted_labels]

        # correct for image scale
        detections[0, :, :4] /= scale

        # visualize detections
        for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
            if score < 0.20:
                continue
            b = detections[0, idx, :4].astype(int)
            cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (0, 0, 255), 3)
            caption = "{} {:.3f}".format(self.csv_generator.label_to_name(label), score)
            cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 0), 3)
            cv2.putText(draw, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)

        return draw

In [5]:
processor = VideoProcessor()

def process_video(input_path, output_path):
    clip = VideoFileClip (input_path)
    result = clip.fl_image(processor.process_image)
    %time result.write_videofile (output_path, audio=False)
    
process_video('test_1.mp4', 'test_1_result.mp4')
process_video('test_2.mp4', 'test_2_result.mp4')

[MoviePy] >>>> Building video test_1_result.mp4
[MoviePy] Writing video test_1_result.mp4


100%|██████████| 246/246 [15:23<00:00,  3.72s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_1_result.mp4 

CPU times: user 1h 45min 51s, sys: 1min 53s, total: 1h 47min 44s
Wall time: 15min 24s
[MoviePy] >>>> Building video test_2_result.mp4
[MoviePy] Writing video test_2_result.mp4


100%|█████████▉| 333/334 [20:51<00:03,  3.76s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_2_result.mp4 

CPU times: user 2h 23min 40s, sys: 2min 28s, total: 2h 26min 9s
Wall time: 20min 53s
